In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pickle
from sklearn.metrics import ConfusionMatrixDisplay
from catboost import CatBoostClassifier 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
def evaluate_model(ytest, ypred, ypred_proba = None):
    if ypred_proba is not None:
        print('ROC-AUC score of the model: {}'.format(roc_auc_score(ytest, ypred_proba[:, 1])))
    print('Accuracy of the model: {}\n'.format(accuracy_score(ytest, ypred)))
    print('Classification report: \n{}\n'.format(classification_report(ytest, ypred)))

In [3]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")['fatality']
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")['fatality']

# CatBoost

In [4]:
clf = CatBoostClassifier(random_state=0, task_type="GPU")
clf.fit(xtrain, ytrain, plot=True, eval_set=(xtest, ytest))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.03582
0:	learn: 0.6163839	test: 0.6164474	best: 0.6164474 (0)	total: 72.7ms	remaining: 1m 12s
1:	learn: 0.5493621	test: 0.5494900	best: 0.5494900 (1)	total: 140ms	remaining: 1m 10s
2:	learn: 0.4917608	test: 0.4919434	best: 0.4919434 (2)	total: 217ms	remaining: 1m 12s
3:	learn: 0.4410079	test: 0.4412408	best: 0.4412408 (3)	total: 287ms	remaining: 1m 11s
4:	learn: 0.3949899	test: 0.3952729	best: 0.3952729 (4)	total: 358ms	remaining: 1m 11s
5:	learn: 0.3556355	test: 0.3559726	best: 0.3559726 (5)	total: 428ms	remaining: 1m 10s
6:	learn: 0.3219595	test: 0.3223444	best: 0.3223444 (6)	total: 498ms	remaining: 1m 10s
7:	learn: 0.2925540	test: 0.2929884	best: 0.2929884 (7)	total: 573ms	remaining: 1m 11s
8:	learn: 0.2662937	test: 0.2667744	best: 0.2667744 (8)	total: 645ms	remaining: 1m 11s
9:	learn: 0.2439964	test: 0.2445218	best: 0.2445218 (9)	total: 717ms	remaining: 1m 10s
10:	learn: 0.2238664	test: 0.2244300	best: 0.2244300 (10)	total: 789ms	remaining: 1m 10s
11:	learn: 

In [5]:
with open('../models/CatBoost.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [6]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/CatBoost.pickle', 'rb') as f:
    clf = pickle.load(f)

In [7]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8477701417312811
Accuracy of the model: 0.9846418193802908

Classification report: 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    797650
           1       0.59      0.01      0.02     12472

    accuracy                           0.98    810122
   macro avg       0.79      0.50      0.50    810122
weighted avg       0.98      0.98      0.98    810122




# Change threshold

In [8]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.014797, G-Mean=0.763
ROC-AUC score of the model: 0.8477701417312811
Accuracy of the model: 0.7663821004737558

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.77      0.87    797650
           1       0.05      0.76      0.09     12472

    accuracy                           0.77    810122
   macro avg       0.52      0.76      0.48    810122
weighted avg       0.98      0.77      0.85    810122




## Check overfitting

In [9]:
ypred = clf.predict(xtrain)
ypred_proba = clf.predict_proba(xtrain)

# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytrain, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytrain,ypred_new_threshold,ypred_proba)

Best Threshold=0.015061, G-Mean=0.766
ROC-AUC score of the model: 0.8508415003872297
Accuracy of the model: 0.7704438343854383

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.77      0.87   3192113
           1       0.05      0.76      0.09     48375

    accuracy                           0.77   3240488
   macro avg       0.52      0.77      0.48   3240488
weighted avg       0.98      0.77      0.86   3240488


